# Early Release Process 
## Introduction 
As part of the prison systems overcrowding resolution process, they are required to release some prisoners that have undertaken at least 40% of their sentence.  This section will look at constructing a model where a list of potential release candidates are selected. Release candidates should match those incoming into the prison system - since releasing someone in high 
supervision when you needed a space in low supervison would not be helpful. 

In [9]:
%pip install -r resources.txt

  Cloning https://github.com/conjure-cp/conjure-notebook.git (to revision v0.0.10) to /tmp/pip-req-build-d0hxxreo
  Running command git clone --filter=blob:none --quiet https://github.com/conjure-cp/conjure-notebook.git /tmp/pip-req-build-d0hxxreo
  Running command git checkout -q d58ac9af77e8c8b8d2c3e9633384fb3670fd03e5
  Resolved https://github.com/conjure-cp/conjure-notebook.git to commit d58ac9af77e8c8b8d2c3e9633384fb3670fd03e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


# Data Loading and Preprocessing 
Next we will handle the data preprocessing and loading into the session.

## 1.1 Load in the data and Run the preprocessing script

In [10]:
import json
import pandas as pd
import sys
sys.path.append('Preprocessing')
from PrisonDataPreprocessor import PrisonDataPreprocessor

# Initialize preprocessor
preprocessor = PrisonDataPreprocessor()

# Load prisoner list
prisoners = preprocessor.load_prisoner_list('Data/PrisonerList.json', key='resident_prisoners')

# Load and process prison template
prison = preprocessor.load_prison_template('Data/PrisonTemplate.json')

# Filter to only occupied beds
prison = prison[prison['occupied'] == True].copy()

# Create prisoners DataFrame and merge with prison data
prisoners_df = pd.DataFrame(prisoners)
merged_df = preprocessor.merge_prisoners_with_beds(prisoners_df, prison)

print(merged_df.head())

  prisoner_id              name   sex  time_served section_id  \
0      P00001     James MacLeod  Male           47         S1   
1      P00002     Robert Fraser  Male           63         S1   
2      P00003  William Campbell  Male           81         S1   
3      P00004     David Stewart  Male           55         S1   
4      P00005       John Murray  Male           72         S1   

         section_name age_category ward_id supervision_level wing_id  \
0  North Wing Section        Adult    S1W1              High   S1W1A   
1  North Wing Section        Adult    S1W1              High   S1W1A   
2  North Wing Section        Adult    S1W1              High   S1W1A   
3  North Wing Section        Adult    S1W1              High   S1W1A   
4  North Wing Section        Adult    S1W1              High   S1W1A   

  cell_type        bed_id  
0    single  S1W1A-C01-B1  
1    single  S1W1A-C02-B1  
2    single  S1W1A-C03-B1  
3    single  S1W1A-C04-B1  
4    single  S1W1A-C05-B1  


You should see in the above example that there is now a mereged prisoner and prison details list which maps the prison bed to the resident prisoner currently assigned to said bed. During this process we have removed any redudndancies that were identified during the merging process so that there are no duplicate columns.

## 1.2 Encoding
Next we will encode the data into an apporpriate format. For this we will use the Ordinal Encoding tool from Sci-kit Learn for mapping numerical values from non-numeric values.


In [11]:
# Encode the merged DataFrame
merged_df_encoded = preprocessor.encode_dataframe(merged_df)
print(f"\nDataFrame after encoding non-numerical data:")
merged_df_encoded.head()


DataFrame after encoding non-numerical data:


,prisoner_id,name,sex,time_served,section_id,section_name,age_category,ward_id,supervision_level,wing_id,cell_type,bed_id
0,0.0,118.0,1.0,47,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,200.0,1.0,63,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,2.0,243.0,1.0,81,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
3,3.0,44.0,1.0,55,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0
4,4.0,126.0,1.0,72,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0


Now that the data has been encoded, lets take a look at the sample data above, we can now see that all values (including those that are redundant for our model) have been encoded. 

## 1.3 Incoming Prisoner load and preprocessing 
Now that we have the resident prisoners and their data handled, lets take a load and encode at the incoming prisoner data which should behave similar to the original prisoner list. 

In [12]:
# Load incoming prisoners to be allocated
prisoners = preprocessor.load_prisoner_list('Data/AllocatedPrisonerList.json', key='incoming_prisoners')

# Encode incoming prisoners using the same encoder
prisoners_encoded = preprocessor.encode_prisoners(prisoners)

print(f"\nFirst prisoner after encoding:")
print(prisoners_encoded[0])


First prisoner after encoding:
{'prisoner_id': 266, 'name': 'Angus "Mad Dog" MacDuff', 'sex': 'Male', 'age_category': 'Adult', 'supervision_level': 'High', 'age_category_encoded': 0, 'sex_encoded': 1, 'supervision_level_encoded': 0}


We now see (at the end of this record) the encoded values for the incoming prisoner which should match the value type of the original prison data values. 

# 2.0 Running the model
## 2.1 Model Preparation 
Just before the model is run, we are requirerd to structure the data in an appropriate way for a successful model run. Below we establish the sets, domains to be used within the model. We also establish the criteria for the minimum time served which can be adjusted as needed. 

In [13]:
# Prepare data for Conjure constraint solver
min_time_served = 40

# Get mappings for early release task (includes time_served)
mappings = preprocessor.prepare_conjure_mappings(
    prisoners_encoded, 
    merged_df_encoded, 
    use_time_served=True
)

# Extract individual mappings for clarity
incoming_prisoners_sex = mappings['incoming_prisoners_sex']
incoming_prisoners_age = mappings['incoming_prisoners_age']
incoming_prisoners_supervision = mappings['incoming_prisoners_supervision']
resident_prison_beds_sex = mappings['bed_sex']
resident_prison_beds_age = mappings['bed_age']
resident_prison_beds_supervision = mappings['bed_supervision']
resident_prison_time_served = mappings['bed_time_served']
min_prisoner_id = mappings['prisoner_min']
max_prisoner_id = mappings['prisoner_max']
min_bed_id = mappings['bed_min']
max_bed_id = mappings['bed_max']

print(f"\nIncoming Prisoners ID Range: {min_prisoner_id} to {max_prisoner_id}")
print(f"Resident Prison Beds ID Range: {min_bed_id} to {max_bed_id}")
print(f"Early Release time_served percentage allowed: {min_time_served}")

# Display the mappings
print(f"\nIncoming Prisoners Sex Mapping: {incoming_prisoners_sex}")
print(f"\nIncoming Prisoners Age Mapping: {incoming_prisoners_age}")
print(f"\nIncoming Prisoners Supervision Mapping: {incoming_prisoners_supervision}")
print(f"\nResident Prisoner Sex Mapping: {resident_prison_beds_sex}")
print(f"\nResident Prisoner Age Mapping: {resident_prison_beds_age}")
print(f"\nResident Prisoner Supervision Mapping: {resident_prison_beds_supervision}")
print(f"\nResident Prisoner Time Served Mapping: {resident_prison_time_served}")


Incoming Prisoners ID Range: 266 to 300
Resident Prison Beds ID Range: 0 to 253
Early Release time_served percentage allowed: 40

Incoming Prisoners Sex Mapping: {266: 1, 267: 1, 268: 0, 269: 1, 270: 1, 271: 0, 272: 1, 273: 1, 274: 0, 275: 0, 276: 1, 277: 1, 278: 1, 279: 1, 280: 1, 281: 1, 282: 0, 283: 1, 284: 0, 285: 1, 286: 0, 287: 1, 288: 0, 289: 1, 290: 0, 291: 1, 292: 0, 293: 1, 294: 0, 295: 1, 296: 0, 297: 1, 298: 0, 299: 1, 300: 0}

Incoming Prisoners Age Mapping: {266: 0, 267: 0, 268: 0, 269: 0, 270: 0, 271: 0, 272: 0, 273: 0, 274: 0, 275: 1, 276: 1, 277: 1, 278: 1, 279: 1, 280: 1, 281: 0, 282: 0, 283: 0, 284: 1, 285: 1, 286: 0, 287: 0, 288: 0, 289: 1, 290: 1, 291: 0, 292: 0, 293: 0, 294: 1, 295: 1, 296: 0, 297: 0, 298: 0, 299: 1, 300: 0}

Incoming Prisoners Supervision Mapping: {266: 0, 267: 0, 268: 0, 269: 2, 270: 2, 271: 2, 272: 1, 273: 1, 274: 1, 275: 0, 276: 1, 277: 1, 278: 1, 279: 1, 280: 1, 281: 0, 282: 2, 283: 1, 284: 0, 285: 1, 286: 0, 287: 2, 288: 1, 289: 1, 290: 0, 

As shown above, we have now prepared the data to be passed to the model, each indice should represent a mapping to the original prisoner(be it incoming or resident) and should be unique to each prisoner. Incoming prisoners indice will just carry over from the last indice found on the resident prisoner list.

## 2.2 Running the model
Now that everything is ready to go, we will now run the model and see what we get out as a result 

In [14]:
%load_ext conjure 

The conjure extension is already loaded. To reload it, use:
  %reload_ext conjure


In [15]:
%%conjure --solver=z3
given min_time_served : int
given min_prisoner_id, max_prisoner_id, min_bed_id, max_bed_id : int
$ Create domains using the ranges
letting Incoming_Prisoners be domain int(min_prisoner_id..max_prisoner_id),
        Resident_Prisoners be domain int(min_bed_id..max_bed_id)

$ Define attribute functions (partial functions, only defined for valid IDs)
given incoming_prisoners_sex, incoming_prisoners_age, incoming_prisoners_supervision: function int --> int
given resident_prison_beds_sex, resident_prison_beds_age, resident_prison_beds_supervision, resident_prison_time_served: function int --> int

$ Decision variable: assign each prisoner to a bed (injective ensures one-to-one mapping)
find early_release: function Incoming_Prisoners --> Resident_Prisoners
such that
    forAll prisoner : Incoming_Prisoners .
        prisoner in defined(early_release),
    
    forAll p1, p2 : Incoming_Prisoners .
        (p1 != p2 /\ p1 in defined(early_release) /\ p2 in defined(early_release)) 
        -> early_release(p1) != early_release(p2),
    $ Prisoners must be assigned to beds that match their sex
    forAll prisoner : Incoming_Prisoners .
        prisoner in defined(early_release) ->
        incoming_prisoners_sex(prisoner) = resident_prison_beds_sex(early_release(prisoner)),
        
    $ Prisoners must be assigned to beds that match their age category
        forAll prisoner : Incoming_Prisoners .
            prisoner in defined(early_release) ->
            incoming_prisoners_age(prisoner) = resident_prison_beds_age(early_release(prisoner)),
        
    $ Prisoners must be assigned to beds that match their supervision level
        forAll prisoner : Incoming_Prisoners .
            prisoner in defined(early_release) ->
            incoming_prisoners_supervision(prisoner) = resident_prison_beds_supervision(early_release(prisoner)),
    
    $ Prisoners can only be assigned to beds where time served is above the minimum threshold 
        forAll prisoner : Incoming_Prisoners .
            prisoner in defined(early_release) ->
            resident_prison_time_served(early_release(prisoner)) >= min_time_served
    


```json
{"early_release": {"266": 12, "267": 2, "268": 19, "269": 24, "270": 37, "271": 138, "272": 89, "273": 72, "274": 107, "275": 158, "276": 172, "277": 232, "278": 185, "279": 219, "280": 189, "281": 9, "282": 134, "283": 74, "284": 163, "285": 169, "286": 16, "287": 116, "288": 108, "289": 246, "290": 165, "291": 13, "292": 133, "293": 75, "294": 199, "295": 239, "296": 15, "297": 68, "298": 95, "299": 242, "300": 52}}
```

| Statistic | Value |
|:-|-:|
| SolverTotalTime | 0.07 |
| SATClauses | 281 |
| SavileRowClauseOut | 0 |
| SavileRowTotalTime | 7.292 |
| SolverSatisfiable | 1 |
| SavileRowTimeOut | 0 |
| SATVars | 36 |


If the process is followed as intended, we will now be able to see an output from the model that shows who should be considered for early release in this instance. But this does not give us suitable information to satisfy that what it has provided is valid. So lets next look to validate the model output

## 2.3 Validation of results
In this stage, we will visually and programatically check that the results the model has produced correctly match the criteria it was given. 

In [16]:
def check_all_erc_match(prisoner, in_prisoner):
    return (prisoner['sex'] == in_prisoner['sex']) and (prisoner['age_category'] == in_prisoner['age_category']) and (prisoner['supervision_level'] == in_prisoner['supervision_level'])and (prisoner['time_served'] >= min_time_served)


valid_selection= True
#Look at the output details of assignments
for i, (prisoner_id, bed_id) in enumerate(early_release.items(), start=1):
    #find the bed_id index location and then get the unencoded details
    er_prisoner_details = merged_df[merged_df_encoded['bed_id'] == bed_id].iloc[0]

    #Get incoming prisoner details
    in_prisoner = next((p for p in prisoners if int(p['prisoner_id'].lstrip('P')) == int(prisoner_id)), None)
    print(f"Early Release to fill for occupant {i}: Prisoner {prisoner_id} -> Bed {bed_id}")
    print(f"Current occupant {er_prisoner_details['name']} to be replaced by incoming prisoner {in_prisoner['name']}")
    print(f"{er_prisoner_details['name']} has served {er_prisoner_details['time_served']}% of sentence.")
    print(f"{er_prisoner_details['name']} details : Sex: {er_prisoner_details['sex']}, Age Category: {er_prisoner_details['age_category']}, Supervision Level: {er_prisoner_details['supervision_level']}, Time Served: {er_prisoner_details['time_served']}%")
    print(f"{in_prisoner['name']} details: Sex: {in_prisoner['sex']}, Age Category: {in_prisoner['age_category']}, Supervision Level: {in_prisoner['supervision_level']}\n")

    # Validate ERC matching
    valid_selection = valid_selection and check_all_erc_match(er_prisoner_details, in_prisoner)

if valid_selection:
    print("All prisoners have been selected by matching the needed requirements to fufil early release")
else: 
    print("There was a mismatch in ERC attributes for some assignments.")

Early Release to fill for occupant 1: Prisoner 266 -> Bed 12
Current occupant Gary Thomson to be replaced by incoming prisoner Angus "Mad Dog" MacDuff
Gary Thomson has served 87% of sentence.
Gary Thomson details : Sex: Male, Age Category: Adult, Supervision Level: High, Time Served: 87%
Angus "Mad Dog" MacDuff details: Sex: Male, Age Category: Adult, Supervision Level: High

Early Release to fill for occupant 2: Prisoner 267 -> Bed 2
Current occupant William Campbell to be replaced by incoming prisoner Bruce McLean
William Campbell has served 81% of sentence.
William Campbell details : Sex: Male, Age Category: Adult, Supervision Level: High, Time Served: 81%
Bruce McLean details: Sex: Male, Age Category: Adult, Supervision Level: High

Early Release to fill for occupant 3: Prisoner 268 -> Bed 19
Current occupant Margaret Wallace to be replaced by incoming prisoner Ailsa Morrison
Margaret Wallace has served 83% of sentence.
Margaret Wallace details : Sex: Female, Age Category: Adult, S

Based on the observation, we can see that the model is correctly assigning early release candidates.